In [42]:
import math
import pyclesperanto_prototype as cle
from skimage.io import imread
import numpy as np 

voxel_size_x_in_microns = 0.1449922
voxel_size_y_in_microns = 0.1449922
voxel_size_z_in_microns = 0.3

deskewing_angle_in_degrees = 30

original_image = imread('./pyclesperanto_prototype/data/RBC_tiny.tif')
original_image = original_image[::4]
voxel_size_z_in_microns = voxel_size_z_in_microns * 4

#Defining Deskewing
scale_factor = 1 #Scale factor is used in clesperanto if we want to scale the images in case they don't fit in memory

#initialise a transform
deskew_transform =cle.AffineTransform3D()
#Shear
shear_factor = math.sin((90 - deskewing_angle_in_degrees) * math.pi / 180.0) * (voxel_size_z_in_microns/voxel_size_y_in_microns)
deskew_transform._matrix[1, 2] = shear_factor
#Scale
new_dz = math.sin(deskewing_angle_in_degrees * math.pi / 180.0) * voxel_size_z_in_microns
scale_factor_z = (new_dz / voxel_size_y_in_microns) * scale_factor
deskew_transform.scale(scale_x=scale_factor, scale_y=scale_factor, scale_z=scale_factor_z)
#Rotate deskewing angle
deskew_transform.rotate(angle_in_degrees = 0 - deskewing_angle_in_degrees, axis=0)
deskew_transform._matrix
deskewed = cle.affine_transform(source= original_image,transform = deskew_transform,auto_size=True)
deskewed = cle.pull(deskewed)
print(deskewed.shape)

(59, 1832, 209)


In [16]:
shear_transform =cle.AffineTransform3D()
#Shear
shear_factor = math.sin((90 - deskewing_angle_in_degrees) * math.pi / 180.0) * (voxel_size_z_in_microns/voxel_size_y_in_microns)
shear_transform._matrix[1, 2] = shear_factor
sheared = cle.affine_transform(source= original_image,transform = shear_transform,auto_size=True)
sheared = cle.pull(sheared)
sheared.shape

(209, 1616, 209)

In [62]:
#scaling and rotation
#Scale
scale_rotate = cle.AffineTransform3D()
new_dz = math.sin(deskewing_angle_in_degrees * math.pi / 180.0) * voxel_size_z_in_microns
scale_factor_z = (new_dz / voxel_size_y_in_microns) * scale_factor
scale_rotate.scale(scale_x=scale_factor, scale_y=scale_factor, scale_z=scale_factor_z)
#Rotate deskewing angle
scale_rotate.rotate(angle_in_degrees = 0 - deskewing_angle_in_degrees, axis=0)
scale_rotate.inverse()

In [33]:

print(sheared[98,798:803,98:103])
crop = sheared[98,798:803,98:103]
coord_sheared = sheared[98,800,100]
coord

x1,y1,z1 = [98,799,99]
x2,y2,z2 = [98,801,101]

print(sheared[x1,y1,z1])
print(sheared[ x2,y2,z2])


[[ 68.  81.  89. 101. 101.]
 [ 72.  87.  94.  99. 103.]
 [ 79.  85.  98. 101. 113.]
 [ 87.  95.  97. 102. 107.]
 [ 86.  85.  93. 103. 101.]]
87.0
102.0


In [54]:
from itertools import product 
neighbours = [list(x)+[1] for x in product((98, 98), (799, 801), (99, 101))][:4]
top_left, top_right, bottom_left,bottom_right = neighbours
neighbours

[[98, 799, 99, 1], [98, 799, 101, 1], [98, 801, 99, 1], [98, 801, 101, 1]]

In [64]:
#neighbours = [list(x)for x in product((98, 98), (799, 801), (99, 101))][0:4]
shear_inverse = shear_transform.copy().inverse()
neighbours_deskewed = np.asarray(list(map(lambda x: shear_inverse._matrix @ x, neighbours)))
neighbours_deskewed


array([[ 98.        ,  89.41825719,  99.        ,   1.        ],
       [ 98.        ,  75.08327249, 101.        ,   1.        ],
       [ 98.        ,  91.41825719,  99.        ,   1.        ],
       [ 98.        ,  77.08327249, 101.        ,   1.        ]])

In [79]:
np.set_printoptions(precision=3)
neighbours_deskewed
shear_img = np.asarray(list(map(lambda x: shear_transform._matrix @ x, neighbours)))
print(shear_img)

[[9.800e+01 1.509e+03 9.900e+01 1.000e+00]
 [9.800e+01 1.523e+03 1.010e+02 1.000e+00]
 [9.800e+01 1.511e+03 9.900e+01 1.000e+00]
 [9.800e+01 1.525e+03 1.010e+02 1.000e+00]]


In [75]:
original_image[98,75:91,99:101]

array([[86, 87],
       [81, 80],
       [83, 79],
       [77, 75],
       [76, 79],
       [75, 86],
       [62, 68],
       [65, 75],
       [72, 65],
       [68, 75],
       [64, 65],
       [61, 69],
       [61, 59],
       [53, 59],
       [52, 62],
       [54, 56]], dtype=uint8)

In [71]:
crop

array([[ 68.,  81.,  89., 101., 101.],
       [ 72.,  87.,  94.,  99., 103.],
       [ 79.,  85.,  98., 101., 113.],
       [ 87.,  95.,  97., 102., 107.],
       [ 86.,  85.,  93., 103., 101.]], dtype=float32)

In [8]:
sheared_image = 
#value at coordinate 
coord = deskewed[25,500,100]
x1 = deskewed[25,500,99]

98.765625

In [12]:
import matplotlib.pyplot as plt
deskewed[25,498:503,98:103]

array([[132.39062 , 113.78516 , 103.609375,  88.40234 ,  85.55469 ],
       [132.60938 , 112.296875, 101.125   ,  82.3125  ,  77.015625],
       [132.9961  , 111.      ,  98.765625,  76.29297 ,  68.48047 ],
       [126.625   , 107.640625,  98.67578 ,  78.640625,  72.03516 ],
       [117.15625 , 103.390625,  99.72656 ,  84.859375,  81.13281 ]],
      dtype=float32)

In [2]:
# Source: https://github.com/QI2lab/OPM/blob/master/reconstruction/image_post_processing.py
# Licensed GPL 3 https://github.com/QI2lab/OPM/blob/master/LICENSE

import sys
import numpy as np
from pathlib import Path
from numba import njit, prange
from functools import partial
import gc


    
# http://numba.pydata.org/numba-doc/latest/user/parallel.html#numba-parallel
@njit(parallel=True)
def deskew(data,theta,distance,pixel_size):
    """
    Perform parallelized orthogonal interpolation into a uniform pixel size grid.
    
    :param data: ndarray
        image stack of uniformly spaced OPM planes
    :param theta: float 
        angle relative to coverslip
    :param distance: float 
        step between image planes along coverslip
    :param pizel_size: float 
        in-plane camera pixel size in OPM coordinates
    :return output: ndarray
        image stack of deskewed OPM planes on uniform grid
    """

    # unwrap parameters 
    [num_images,ny,nx]=data.shape     # (pixels)

    # change step size from physical space (nm) to camera space (pixels)
    pixel_step = distance/pixel_size    # (pixels)

    # calculate the number of pixels scanned during stage scan 
    scan_end = num_images * pixel_step  # (pixels)

    # calculate properties for final image
    final_ny = np.int64(np.ceil(scan_end+ny*np.cos(theta*np.pi/180))) # (pixels)
    final_nz = np.int64(np.ceil(ny*np.sin(theta*np.pi/180)))          # (pixels)
    final_nx = np.int64(nx)                                           # (pixels)

    # create final image
    output = np.zeros((final_nz, final_ny, final_nx),dtype=np.float32)  # (time, pixels,pixels,pixels - data is float32)

    # precalculate trig functions for scan angle
    tantheta = np.float32(np.tan(theta * np.pi/180)) # (float32)
    sintheta = np.float32(np.sin(theta * np.pi/180)) # (float32)
    costheta = np.float32(np.cos(theta * np.pi/180)) # (float32)

    # perform orthogonal interpolation

    # loop through output z planes
    # defined as parallel loop in numba
    # http://numba.pydata.org/numba-doc/latest/user/parallel.html#numba-parallel
    for z in prange(0,final_nz):
        # calculate range of output y pixels to populate
        y_range_min=np.minimum(0,np.int64(np.floor(np.float32(z)/tantheta)))
        y_range_max=np.maximum(final_ny,np.int64(np.ceil(scan_end+np.float32(z)/tantheta+1)))

        # loop through final y pixels
        # defined as parallel loop in numba
        # http://numba.pydata.org/numba-doc/latest/user/parallel.html#numba-parallel
        for y in prange(y_range_min,y_range_max):

            # find the virtual tilted plane that intersects the interpolated plane 
            virtual_plane = y - z/tantheta

            # find raw data planes that surround the virtual plane
            plane_before = np.int64(np.floor(virtual_plane/pixel_step))
            plane_after = np.int64(plane_before+1)

            # continue if raw data planes are within the data range
            if ((plane_before>=0) and (plane_after<num_images)):
                
                # find distance of a point on the  interpolated plane to plane_before and plane_after
                l_before = virtual_plane - plane_before * pixel_step
                l_after = pixel_step - l_before
                
                # determine location of a point along the interpolated plane
                za = z/sintheta
                virtual_pos_before = za + l_before*costheta
                virtual_pos_after = za - l_after*costheta

                # determine nearest data points to interpoloated point in raw data
                pos_before = np.int64(np.floor(virtual_pos_before))
                pos_after = np.int64(np.floor(virtual_pos_after))

                # continue if within data bounds
                if ((pos_before>=0) and (pos_after >= 0) and (pos_before<ny-1) and (pos_after<ny-1)):
                    
                    # determine points surrounding interpolated point on the virtual plane 
                    dz_before = virtual_pos_before - pos_before
                    dz_after = virtual_pos_after - pos_after

                    # compute final image plane using orthogonal interpolation
                    output[z,y,:] = (l_before * dz_after * data[plane_after,pos_after+1,:] +
                                    l_before * (1-dz_after) * data[plane_after,pos_after,:] +
                                    l_after * dz_before * data[plane_before,pos_before+1,:] +
                                    l_after * (1-dz_before) * data[plane_before,pos_before,:]) /pixel_step


    return output


deskewed = deskew(original_image,deskewing_angle_in_degrees,voxel_size_z_in_microns,voxel_size_x_in_microns)
deskewed.shape

(59, 1832, 209)

(3, 6, 9)